In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
device = torch.device('cpu')

In [20]:
'''As per the defination provided on pytorch : https://pytorch.org/docs/stable/_modules/torch/utils/data/sampler.html
I have created a Custom batch sample following the same implementation on above link.
Below implementation of sampler is taken from forum https://discuss.pytorch.org/t/load-the-same-number-of-data-per-class/65198/3'''
class CustomBatchSampler(BatchSampler):

    def __init__(self, dataset, n_classes, n_samples):
        loader = DataLoader(dataset)
        self.labels_list = []
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset)

In [21]:
# MNIST dataset 
#load training data set.
train_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)
#Load testing data set.
test_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [22]:
'''Checking the data set classes and amount of data in each class.'''
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    
    for element in dataset_obj:
        y_lbl = element[1]
        y_lbl = idx2class[y_lbl]
        count_dict[y_lbl] += 1
            
    return count_dict
print("Distribution of classes: \n", get_class_distribution(train_dataset))

Distribution of classes: 
 {'T-shirt/top': 6000, 'Trouser': 6000, 'Pullover': 6000, 'Dress': 6000, 'Coat': 6000, 'Sandal': 6000, 'Shirt': 6000, 'Sneaker': 6000, 'Bag': 6000, 'Ankle boot': 6000}


In [23]:
input_size = 784 # 28x28
hidden_size = 500 
total_classes = 10
epochs_count = 2
batch_size = 100
learning_rate = 0.001

In [24]:
'''For different configuration just uncomment below line and execute all cells.'''
batch_sampler = CustomBatchSampler(train_dataset, total_classes, 100)
#batch_sampler = CustomBatchSampler(train_dataset, total_classes, 500)
#batch_sampler = CustomBatchSampler(train_dataset, total_classes, 1000)
#batch_sampler = CustomBatchSampler(train_dataset, total_classes, 5000)

In [25]:
# Load the data.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_sampler=batch_sampler)

'''For running on the full dataset, uncomment this line and comment above line.'''
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [26]:
# MLP with 2 layers
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, total_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.n_images_per_class = 1000
        self.n_classes = 10
        self.l1 = torch.nn.Linear(input_size, hidden_size) 
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(hidden_size, total_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [27]:
nnmodel = NeuralNet(input_size, hidden_size, total_classes).to(device)

In [28]:
# Using Cross Entropy Loss.
cross_entropy_loss = nn.CrossEntropyLoss()
#Using Adam optimizer
optimizer = torch.optim.Adam(nnmodel.parameters(), lr=learning_rate)  


In [29]:
# Training the NN
steps = len(train_loader)
for epoch in range(epochs_count):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = nnmodel(images)
        loss = cross_entropy_loss(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{epochs_count}], Step [{i+1}/{steps}], Loss: {loss.item():.4f}')


In [30]:
# Test the NN
with torch.no_grad():
    nn_correct = 0
    nn_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = nnmodel(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        nn_samples += labels.size(0)
        nn_correct += (predicted == labels).sum().item()

    acc = 100.0 * nn_correct / nn_samples
    print(f'Accuracy of the network on test images: {acc} %')


Accuracy of the network on test images: 83.52 %


In [31]:
# Classification Report
print(classification_report(labels, predicted, zero_division=1))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72        11
           1       0.92      0.92      0.92        12
           2       0.33      0.50      0.40         6
           3       1.00      1.00      1.00         7
           4       0.86      0.75      0.80         8
           5       1.00      0.82      0.90        11
           6       0.60      0.25      0.35        12
           7       0.88      1.00      0.93         7
           8       0.88      1.00      0.93        14
           9       0.92      1.00      0.96        12

    accuracy                           0.81       100
   macro avg       0.80      0.81      0.79       100
weighted avg       0.82      0.81      0.80       100



In [32]:
# Confusion matrix
print(confusion_matrix(labels,predicted))

[[ 9  0  0  0  0  0  1  0  1  0]
 [ 0 11  1  0  0  0  0  0  0  0]
 [ 1  1  3  0  0  0  0  0  1  0]
 [ 0  0  0  7  0  0  0  0  0  0]
 [ 0  0  1  0  6  0  1  0  0  0]
 [ 0  0  0  0  0  9  0  1  0  1]
 [ 4  0  4  0  1  0  3  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0]
 [ 0  0  0  0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0  0  0  0 12]]
